<a href="https://colab.research.google.com/github/fitrianrr/magang/blob/main/Ini_nyoba_TF_IDF_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
path = "/content/drive/MyDrive/Magang DLA Fitria Nur Rahmadani /code-dataset.xlsx"
data = pd.read_excel(path)
data.head()

,subject,kodefikasi
0,Build Platform Otomasi Testing Respon AI,DLA-F2
1,ML Playground - Prediksi,DLA-F2
2,Tambahan Penetapan Mahasiswa Gel 1 2024 11 Jan...,DLA-F1
3,Tambahan Plotting Penetapan Mahasiswa PPG Praj...,DLA-F1
4,Proses Migrasi Data email Belajar.id Mahasiswa...,DLA-F7


In [5]:
df = data.dropna().reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370 entries, 0 to 369
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   subject     370 non-null    object
 1   kodefikasi  370 non-null    object
dtypes: object(2)
memory usage: 5.9+ KB


In [6]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')


# Fungsi utama untuk membersihkan teks
def clean_text(text):
    text = text.lower() # Ubah menjadi huruf non kapital semua
    text = re.sub(r"\\t|\\n|\\u|\\", " ", text)  # Hapus tab, newline, dan backslash
    text = text.encode('ascii', 'replace').decode('ascii')  # Hapus karakter non-ASCII
    text = re.sub(r"([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)", " ", text)  # Hapus mention, hashtag, dan link
    text = text.replace("http://", " ").replace("https://", " ")  # Hapus URL yang tidak lengkap
    text = re.sub(r"\d+", "", text)  # Hapus angka
    text = text.translate(str.maketrans("", "", string.punctuation))  # Hapus tanda baca
    text = re.sub(r"\b[a-zA-Z]\b", "", text)  # Hapus karakter tunggal
    text = re.sub(r"\s+", " ", text).strip()  # Hapus spasi ganda dan spasi di awal/akhir

    return text

# TMenerapkan fungsi tersebut pada kolom 'subject' beserta tokenisasi
df['cleaned_text'] = df['subject'].apply(lambda x: word_tokenize(clean_text(x)))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
pip install Sastrawi

In [8]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Mengembalikan tiap kata ke kata dasarnya
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Menerapkan fungsi tersebut pada kolom 'cleaned_text'
df['cleaned_text'] = df['cleaned_text'].apply(lambda x: [stemmer.stem(word) for word in x])

In [9]:
# Fungsi untuk menghapus stopwords
def rem_stopw(text):
    listStopword = set(stopwords.words('indonesian'))
    return [word for word in text if word not in listStopword]

In [10]:
# Menerapkan fungsi rem_stopw pada kolom 'cleaned_text'
df['cleaned_text'] = df['cleaned_text'].apply(rem_stopw)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=8, min_df=5, max_df=0.7, stop_words=stopwords.words('indonesian'))
X = tfidfconverter.fit_transform(df['cleaned_text'].apply(lambda x: ' '.join(x))).toarray()

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, df['kodefikasi'], test_size=0.2, random_state=0)

In [13]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier().fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(y_pred)

['DLA-F6' 'DLA-F1' 'DLA-F2' 'DLA-F7' 'DLA-F7' 'DLA-F1' 'DLA-F2' 'DLA-F7'
 'DLA-F1' 'DLA-F1' 'DLA-F7' 'DLA-F2' 'DLA-F1' 'DLA-F6' 'DLA-F7' 'DLA-F2'
 'DLA-F1' 'DLA-F1' 'DLA-F7' 'DLA-F7' 'DLA-F7' 'DLA-F1' 'DLA-F2' 'DLA-F1'
 'DLA-F1' 'DLA-F1' 'DLA-F2' 'DLA-F7' 'DLA-F7' 'DLA-F1' 'DLA-F2' 'DLA-F1'
 'DLA-F1' 'DLA-F6' 'DLA-F7' 'DLA-F7' 'DLA-F1' 'DLA-F7' 'DLA-F1' 'DLA-F2'
 'DLA-F1' 'DLA-F2' 'DLA-F2' 'DLA-F2' 'DLA-F7' 'DLA-F2' 'DLA-F3' 'DLA-F1'
 'DLA-F2' 'DLA-F1' 'DLA-F1' 'DLA-F2' 'DLA-F7' 'DLA-F3' 'DLA-F1' 'DLA-F2'
 'DLA-F1' 'DLA-F7' 'DLA-F1' 'DLA-F7' 'DLA-F1' 'DLA-F1' 'DLA-F7' 'DLA-F6'
 'DLA-F7' 'DLA-F2' 'DLA-F1' 'DLA-F2' 'DLA-F1' 'DLA-F7' 'DLA-F1' 'DLA-F1'
 'DLA-F2' 'DLA-F2']


In [14]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))

[[24  1  0  0  0  1  1  0]
 [ 0  5  0  0  0  0  0  0]
 [ 0  4  1  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0]
 [ 0  3  1  0  0  1  0  0]
 [ 1  0  0  0  0  2  0  0]
 [ 2  2  0  0  0  0 19  0]
 [ 2  3  0  0  0  0  0  0]]


In [15]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      DLA-F1       0.83      0.89      0.86        27
      DLA-F2       0.26      1.00      0.42         5
      DLA-F3       0.50      0.20      0.29         5
      DLA-F4       0.00      0.00      0.00         1
      DLA-F5       0.00      0.00      0.00         5
      DLA-F6       0.50      0.67      0.57         3
      DLA-F7       0.95      0.83      0.88        23
      DLA-F8       0.00      0.00      0.00         5

    accuracy                           0.69        74
   macro avg       0.38      0.45      0.38        74
weighted avg       0.67      0.69      0.66        74



In [16]:
print(accuracy_score(y_test, y_pred))

0.6891891891891891
